# PyTorch GPU Benchmark: House Prices

This notebook is designed to verify and showcase GPU acceleration (CUDA) using PyTorch. It uses a subset of the Ames House Prices dataset to perform a regression task.

### Objectives:
1. **GPU Verification**: Detect and utilize CUDA devices.
2. **Data Pipeline**: Load, filter, and standardize data using Pandas and PyTorch.
3. **Performance**: Execute a high-epoch training loop to demonstrate the speed of GPU processing once initialized.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import time

# 1. Setup Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps") if torch.backends.mps.is_available() else device
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

Using device: cpu


In [ ]:
# 2. Load Data
df = pd.read_csv('../data/house_prices/house_prices_train.csv')

features = ["GrLivArea", "OverallQual", "GarageCars", "FullBath"]
target = "SalePrice"

X = df[features].values.astype(np.float32)
y = df[target].values.astype(np.float32).reshape(-1, 1)

# 3. Standardization (Manual implementation for GPU-readiness)
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / (X_std + 1e-8)

y_mean = y.mean()
y_std = y.std()
y = (y - y_mean) / (y_std + 1e-8)

# Convert to Tensors and move to Device
X_tensor = torch.tensor(X).to(device)
y_tensor = torch.tensor(y).to(device)

print(f"Data moved to {device}. Shape: {X_tensor.shape}")

Data moved to cpu. Shape: torch.Size([1460, 4])


In [ ]:
# 4. Define an Intricate MLP for Benchmarking
class BenchmarkModel(nn.Module):
    def __init__(self, input_dim):
        super(BenchmarkModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
        
    def forward(self, x):
        return self.net(x)

model = BenchmarkModel(len(features)).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(model)

BenchmarkModel(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=1, bias=True)
  )
)


In [ ]:
# 5. Training Loop with Timing
epochs = 5000
print(f"Starting training for {epochs} epochs...")

start_time = time.time()

model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 500 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

end_time = time.time()
total_time = end_time - start_time

print("\nTraining complete!")
print(f"Total Time on {device}: {total_time:.2f} seconds")
print(f"Average Time per Epoch: {(total_time/epochs)*1000:.4f} ms")

Starting training for 5000 epochs...
Epoch [500/5000], Loss: 0.0939
Epoch [1000/5000], Loss: 0.0793
Epoch [1500/5000], Loss: 0.0761
Epoch [2000/5000], Loss: 0.0686
Epoch [2500/5000], Loss: 0.0614
Epoch [3000/5000], Loss: 0.0598
Epoch [3500/5000], Loss: 0.0551
Epoch [4000/5000], Loss: 0.0533
Epoch [4500/5000], Loss: 0.0515
Epoch [5000/5000], Loss: 0.0458

Training complete!
Total Time on cpu: 86.37 seconds
Average Time per Epoch: 17.2730 ms


### Using M1 Air GPU (8 cores)

Training complete!
Total Time on mps: 42.88 seconds
Average Time per Epoch: 8.5755 ms

### Using M1 Air CPU (8 cores)

Training complete!
Total Time on cpu: 86.37 seconds
Average Time per Epoch: 17.2730 ms

In [ ]:
# 6. Verify Results
model.eval()
with torch.no_grad():
    sample_idx = 0
    prediction = model(X_tensor[sample_idx].unsqueeze(0))
    pred_actual = (prediction.item() * y_std) + y_mean
    true_actual = (y_tensor[sample_idx].item() * y_std) + y_mean
    
    print("\nSample Prediction (ID 1):")
    print(f"Predicted SalePrice: ${pred_actual:,.2f}")
    print(f"True SalePrice:      ${true_actual:,.2f}")


Sample Prediction (ID 1):
Predicted SalePrice: $167,681.78
True SalePrice:      $208,500.00
